## Sofiia Druchyna
## CMPS 3160: Intro to Data Science 
## Who Do Threat Actors Target in the Cyberspace? What Are They Looking For?

In [8]:
!ls

DSProject.ipynb      FullAttackList.csv   FullThreatActors.csv


In this project, I am planning to use CISSM Cyber Events Dataset. <br>
The information in the dataset brings together open-source information surrounding a range of publically acknowledged cyber events on private and public organizations. <br> 
Events from 2014 through present have been coded to standardize information on threat actor, threat actor country, motive, target, end effects, industry, and country of impact.

In [2]:
import pandas as pd

df_threats = pd.read_csv('./FullThreatActors.csv')
display(df_threats)

,Name,Country,Actor Type,Established,Last Seen
0,$2a$45,Undetermined,Criminal,NaN,2018-03-09
1,'desserped',Undetermined,Criminal,NaN,2023-10-11
2,0mega,Undetermined,Criminal,NaN,2023-02-20
3,0x0D1337,Undetermined,Undetermined,NaN,2015-09-30
4,0x2Taylor,Undetermined,Hacktivist,NaN,2016-10-13
...,...,...,...,...,...
95,Angel_Of_Truth,Canada,Hacktivist,NaN,2015-04-21
96,AnibaLeaks,Undetermined,Criminal,NaN,2021-09-30
97,Anka Neferler Timi,France,Hacktivist,NaN,2020-12-22
98,AnoaGhost,Undetermined,Undetermined,NaN,2018-04-17


In [6]:
df_threats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         100 non-null    object 
 1   Country      100 non-null    object 
 2   Actor Type   100 non-null    object 
 3   Established  0 non-null      float64
 4   Last Seen    100 non-null    object 
dtypes: float64(1), object(4)
memory usage: 4.0+ KB


In the first dataset above, we can see all detected Threat Actors.

- Question 1. What are the trends among the cyber threat actors (if there are any) regarding the country origin?
- Question 2. What are the trends among the cyber threat actors regarding their type? 
- Question 3. Were the same threat actors seen multiple times?
- Question 4. Did multiple threat actors act around the same time? If so, does it happen more often that not?

In [3]:
df_attack = pd.read_csv('./FullAttackList.csv')
df_attack

,Date,Location,Victim,Industry,Actor Location,Actor,Motive,Type,Sub-Type
0,2023-10-31,United States of America,Smoothie King,Accommodation and Food Services,Undetermined,Undetermined,Financial,Exploitive,Undetermined
1,2023-10-31,Italy,Avangate,Other Services (except Public Administration),Undetermined,Alpha Team,Financial,Mixed,Data Attack
2,2023-10-30,United States of America,Dallas County,Public Administration,Undetermined,PLAY,Financial,Mixed,Data Attack
3,2023-10-30,Russian Federation,National Payment Card System (NSPK),Finance and Insurance,Ukraine,DumpForums and Ukrainian Cyber Alliance,Protest,Disruptive,Message Manipulation
4,2023-10-30,Germany,Südwestfalen IT,"Professional, Scientific, and Technical Services",Undetermined,Undetermined,Financial,Mixed,Data Attack
...,...,...,...,...,...,...,...,...,...
295,2023-09-12,United States of America,East Jackson Community Schools,Educational Services,Undetermined,Undetermined,Undetermined,Undetermined,Undetermined
296,2023-09-12,Colombia,IFX Networks Colombia,Information,Undetermined,Ransom House,Financial,Mixed,Data Attack
297,2023-09-12,Italy,SDA,Transportation and Warehousing,Italy,ADHD,Protest,Mixed,Data Attack
298,2023-09-12,United States of America,Fauquier County Public Schools,Educational Services,Undetermined,LockBit 3.0,Financial,Mixed,Data Attack


In [4]:
df_attack.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            300 non-null    object
 1   Location        300 non-null    object
 2   Victim          300 non-null    object
 3   Industry        299 non-null    object
 4   Actor Location  300 non-null    object
 5   Actor           300 non-null    object
 6   Motive          300 non-null    object
 7   Type            300 non-null    object
 8   Sub-Type        300 non-null    object
dtypes: object(9)
memory usage: 21.2+ KB


In [5]:
df_attack.describe()

,Date,Location,Victim,Industry,Actor Location,Actor,Motive,Type,Sub-Type
count,300,300,300,299,300,300,300,300,300
unique,49,36,299,18,10,58,6,4,11
top,2023-09-15,United States of America,Manchester Airport,Health Care and Social Assistance,Undetermined,Undetermined,Financial,Exploitive,Data Attack
freq,21,158,2,65,242,113,222,128,114


In the next dataset, we can see all detected CyberAttacks. This dataset is especially interesting for the data analysis as there are several characteristics that we can try to find
correlation among. <br>
- Question 1. When did most attacks occur?
- Question 2. What was the most common victim? Did those victims vary by their income?
- Question 3. What industry was targeted the most?
- Question 4. What are some of the most common actor locations?
- Question 5. What are some of the most common motives?
- Question 6. What are some of the most common attack types? And what were the sub-types of those attacks?
- Question 7. Is there a correlation among all these variables? 


In the future milestones, I am planning to provide visualization on the various categories of attackers and cyber crimes and see the existing trends in the ongoing cybercrimes. <br>
Further, it would be interesting to track which attackers caused the most impactful harm to their victims and try to identify the main aspects that led to that conclusion. <br>

One of the future steps would also try to dive into the specic category of victims (e.g. specific Industry sector) and analyze the occuring cybercrimes and their trends. <br> 
(Potentially will choose the healthcare dataset:)